In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser

In [6]:
## Load the textfile
loader = TextLoader('langchain_sample.txt')
raw_documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)

docs = text_splitter.split_documents(raw_documents)

In [7]:
docs

[Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='Retrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM responses. LangChain makes it easy to implement RAG using vector databases like FAISS, Chroma, and Pinecone.\nBM25 is a traditional 

In [8]:
## User query
query = "How can i use Langchain to build an application with memory and tools?"

In [9]:
### FAISS and Huggingface model for embeddings
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorestore=FAISS.from_documents(docs, embedding_model)
retriever = vectorestore.as_retriever(search_kwargs={'k': 8})

In [35]:
## Promptand use the llm
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="gemma3:4b-it-q4_K_M",
    num_ctx=8192
)

In [36]:
# Prompt template
prompt = PromptTemplate.from_template("""
You are a helpful assistant. Your task is to rantk thefollowing documents from most to least relevant to the user's question.

User question: "{question}"

Documents: 
{documents}

Instructions:
- Think about the relevance of each document to the user's question.
- return a list of documents indices in ranked order, starting from the most relevant.

Output format: comma-sepatated document indices (e.g., 2,1,3,0,...)
""")

In [37]:
## Retrieval chain
retrieved_docs = retriever.invoke(query)
retrieved_docs

[Document(id='effb3334-2100-488b-95b8-e3276cf6e06e', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='f2e8263b-963d-4097-938b-94927a252d14', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='ec3fcf03-84a3-4d56-8400-00f5c47b9e19', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging F

In [38]:
chain = prompt | llm | StrOutputParser()
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Your task is to rantk thefollowing documents from most to least relevant to the user\'s question.\n\nUser question: "{question}"\n\nDocuments: \n{documents}\n\nInstructions:\n- Think about the relevance of each document to the user\'s question.\n- return a list of documents indices in ranked order, starting from the most relevant.\n\nOutput format: comma-sepatated document indices (e.g., 2,1,3,0,...)\n')
| ChatOllama(model='gemma3:4b-it-q4_K_M', num_ctx=8192)
| StrOutputParser()

In [39]:
docs_line = [f"{i+1}. {doc.page_content}" for i, doc in enumerate(retrieved_docs)]
formatted_docs = "\n".join(docs_line)

In [40]:
response = chain.invoke({"question": query, "documents": formatted_docs})
print(response)

2,1,5,4,3,6 



In [41]:
# Step 5: parse and rerank
indices = [int(x.strip()) - 1 for x in response.split(",") if x.strip().isdigit()]
indices

[1, 0, 4, 3, 2, 5]

In [42]:
reranked_docs = [retrieved_docs[i] for i in indices if 0 <= i < len(retrieved_docs)]
reranked_docs

[Document(id='f2e8263b-963d-4097-938b-94927a252d14', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='effb3334-2100-488b-95b8-e3276cf6e06e', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='fb8866be-d52d-4144-ac3b-b0f43f542221', metadata={'source': 'langchain_sample.txt'}, page_content='Retrieval-Augmented Generation (RAG) is a powerful technique where external k

In [44]:
# Step 6: Show results
print("\n Final Reranked Results:")
for i, doc in enumerate(reranked_docs, 1):
    print(f"\nRank {i}:\n{doc.page_content}")


 Final Reranked Results:

Rank 1:
LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.

Rank 2:
LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.
Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.

Rank 3:
Retrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM responses. LangChain makes it easy to implement RAG using vector databases like FAISS, Chroma, and Pinecone.
BM25 is a traditional sparse retrieval method that scores documents based on keyword matching. Although fast, it often